In [1]:
!pip install transformers einops accelerate langchain bitsandbytes sentence_transformers pypdf python-dotenv

  Using cached transformers-4.35.2-py3-none-any.whl.metadata (123 kB)
  Using cached einops-0.7.0-py3-none-any.whl.metadata (13 kB)
  Using cached accelerate-0.25.0-py3-none-any.whl.metadata (18 kB)
  Using cached langchain-0.0.346-py3-none-any.whl.metadata (16 kB)
  Using cached bitsandbytes-0.41.2.post2-py3-none-any.whl.metadata (9.8 kB)
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Using cached pypdf-3.17.1-py3-none-any.whl.metadata (7.5 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
  Using cached filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached huggingface_hub-0.19.4-py3-none-any.whl.metadata (14 kB)
  Using cached tokenizers-0.15.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached safetensors-0.4.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached torch-2.1.1-cp39-cp39-manylinux1_x86_64.whl.metadata (25 kB)
  Using cached SQLAlchemy-2.0.23-cp39-cp39-ma

In [4]:
!pip install typing-extensions==4.5.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydantic 2.5.2 requires typing-extensions>=4.6.1, but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.14.5 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.


In [10]:
!pip install accelerate

In [2]:
!pip install llama-index==0.8.5

  Using cached tiktoken-0.5.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached openai-1.3.7-py3-none-any.whl.metadata (17 kB)
  Using cached pandas-2.1.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 1.8 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 3.1 MB/s eta 0:00:00a 0:00:01
  Using cached distro-1.8.0-py3-none-any.whl (20 kB)
  Using cached httpx-0.25.2-py3-none-any.whl.metadata (6.9 kB)
  Using cached pytz-2023.3.post1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Using cached httpcore-1.0.2-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 678.4/678.4 kB 11.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 17.8 MB/s eta 0:00:00:00:0100:01


In [4]:
import logging
import sys
import torch
from pprint import pprint
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
# from llama_index import LangchainEmbedding
from llama_index.prompts.prompts import SimpleInputPrompt

In [5]:
from pathlib import Path
from llama_index import download_loader

In [6]:
PDFReader = download_loader("PDFReader")
loader = PDFReader()

In [7]:
system_prompt = "You are a data extractor. Extract the exact data from given document. If no information is found, please reply 'No information found'"
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [8]:
context_window = 4096
temperature = 0.1
model_name = 'litelo/llama-2-case-whisper'

In [9]:
from transformers import LlamaTokenizer, LlamaForCausalLM
tokenizer = LlamaTokenizer.from_pretrained(model_name)
model_llm = LlamaForCausalLM.from_pretrained(
    model_name,
    # torch_dtype=torch.float16,
    # load_in_4bit=True,
    device_map='auto'
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [50]:
llm = HuggingFaceLLM(
    context_window=context_window,
    max_new_tokens=256,
    generate_kwargs={"temperature":temperature,"top_p":0.5, "do_sample": False},
    system_prompt= system_prompt,
    query_wrapper_prompt = query_wrapper_prompt,
    tokenizer=tokenizer,
    model=model_llm,
    device_map='auto',
    model_kwargs={"temperature":0.1, "top_p":0.5,"use_auth_token": True}
)

In [51]:
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [52]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [54]:
query_engine = index.as_query_engine()

while 1:
    user_query=input("Question:")
    if user_query=='quit':
        break

    response=query_engine.query(user_query)
    print("Answer: ", response.response, "\n")

Question: what is this case about


/root/hackathon/oneapi-devsummit-sea-2023/itex_cpu/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Answer:  This case appears to be about the continuation of a rehabilitation center for people with disabilities in the states of West Bengal and Karnataka. The center was started by the Central Government under a scheme, but the funds for the project have been exhausted and the state governments are not willing to continue the project. The case involves issues related to the responsibility of the state governments in implementing the Persons with Disabilities (Equal Opportunities, Protection of Rights and Full Participation) Act, 1995, and the control of the rehabilitation centers for the benefit of the people for whom they were started. 



Question: exit


Answer:  The exact data extracted from the given document is:

* Page label: 6
* File name: 9995-2002___jonew__judis__28251.pdf
* Context information: The document is a Supreme Court of India judgment, and the extracted data refers to the pages and file name of the document. 



Question: qui


KeyboardInterrupt: 

In [12]:
import json
from flask import Flask, request, jsonify

app = Flask(__name__)


In [13]:
@app.route('/chat', methods=['POST'])
def get():
    json_data = request.json
    query = json_data["query"]
    context = json_data["context"]
    if context!="" :
        documents = loader.load_data(file=Path('../supreme-court-data/data/data/'+context))
        index = VectorStoreIndex.from_documents(documents, service_context=service_context)
        query_engine = index.as_query_engine()
        response=query_engine.query(query)
        json_data['response'] = response.response
        return jsonify(json_data)
    else:
        response = llm.complete(query)
        json_data['response'] = response.text
        return jsonify(json_data)

In [ ]:
app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
